In [2]:
import warnings
warnings.filterwarnings('ignore')
directory = 'new-c1'
import nltk

In [2]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
#python -m nltk.downloader all

SyntaxError: invalid syntax (<ipython-input-3-a98312d574ff>, line 1)

In [23]:
import os
directory = "./new-c1/"
raw_docs = []
for filename in os.listdir(directory):
    try:
        f=open("./new-c1/"+ filename, "r")
        text = f.read()
        raw_docs.append(text)
    except Exception:
        print('didnt read paper' +filename)
    f.close()

didnt read paper.DS_Store
didnt read paper.ipynb_checkpoints


In [24]:
# tokenizing the words
print('started')
from nltk.tokenize import word_tokenize
tokenized_docs = [word_tokenize(doc.lower()) for doc in raw_docs]
print('done')

started
done


In [25]:
#removing punctuation and numbers
import re
import string
regex = re.compile('[%s]' % re.escape(string.punctuation)) #see documentation here: http://docs.python.org/2/library/string.html

tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    new_review = []
    for token in review:
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_no_punctuation.append(new_review)
print('done')

done


In [26]:
# I am trying to find the most comonly used words in a research paper, so that I can remove them when cleaning the Data
f = open('firstword.txt', 'r')
text = f.read()
f.close()
split= nltk.tokenize.sent_tokenize(text)
firstwords = []

for sentence in split:
    arrayOFsentence = sentence.split()
    firstwords.append(arrayOFsentence[:3])   

noSYMz =[]
for sentence in firstwords:
    for word in sentence:
        if word.isalpha() and word[0].isupper():
            noSYMz.append(word.lower())

In [27]:
# removing common words; new_review has words with no punctuation
from nltk.corpus import stopwords

tokenized_docs_no_stopwords = []

for doc in tokenized_docs_no_punctuation:
    new_term_vector = []
    for word in doc:
        if not word in stopwords.words('english') and word not in noSYMz:
            new_term_vector.append(word)
    
    tokenized_docs_no_stopwords.append(new_term_vector)

In [28]:
import nltk
withtag4elm = [nltk.pos_tag(paper) for paper in tokenized_docs_no_stopwords]

In [29]:
# Lemmatizing words with POS Tag

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
lematizer = WordNetLemmatizer()
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return wordnet.NOUN

preprocessed_docs = []
for doc in withtag4elm:  
    wordstotheirROOT = []
    for word,tag in doc:
        wordstotheirROOT.append(lematizer.lemmatize(word, get_wordnet_pos(tag)))
    preprocessed_docs.append(wordstotheirROOT)

removing3z = []
for doc in preprocessed_docs: 
    no3 = []
    for word in doc:
        #print(word)
        if len(word) > 3 and word.isalpha():
            no3.append(word)
    removing3z.append(no3)

# no stopwords has the cleaned stuff, also here i am converting the words to their root word - removing the ing, es, s etc.
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
lematizer = WordNetLemmatizer()

lemmitized_words = []

for word in no_stopwords:
    lemmitized_words.append(lematizer.lemmatize(word))
print(lemmitized_words[:20])

In [30]:
print(lematizer.lemmatize("predicting"))

predicting


In [31]:
from gensim import corpora, models

list_of_list_of_tokens = removing3z
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

num_topics = 20
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

CPU times: user 15.9 s, sys: 242 ms, total: 16.2 s
Wall time: 9.96 s


In [34]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(str(i)+": "+ topic)
    print()

#print(len(tokenized_docs_no_punctuation))

0: 0.009*"system" + 0.007*"memory" + 0.007*"performance" + 0.007*"xeon" + 0.006*"time" + 0.005*"intel" + 0.005*"learn" + 0.005*"process" + 0.005*"node" + 0.005*"compute"

1: 0.008*"performance" + 0.007*"application" + 0.006*"time" + 0.006*"system" + 0.006*"code" + 0.006*"memory" + 0.005*"show" + 0.005*"result" + 0.004*"parallel" + 0.004*"compute"

2: 0.024*"sensor" + 0.011*"quantum" + 0.009*"polygon" + 0.008*"point" + 0.007*"network" + 0.007*"state" + 0.007*"photon" + 0.006*"coverage" + 0.006*"ﬁnger" + 0.005*"measure"

3: 0.009*"time" + 0.008*"system" + 0.007*"network" + 0.006*"application" + 0.005*"performance" + 0.005*"compute" + 0.005*"transfer" + 0.005*"show" + 0.005*"process" + 0.004*"work"

4: 0.010*"code" + 0.009*"system" + 0.009*"task" + 0.009*"user" + 0.008*"module" + 0.008*"program" + 0.007*"software" + 0.006*"time" + 0.005*"compute" + 0.005*"core"

5: 0.010*"system" + 0.010*"power" + 0.009*"grid" + 0.008*"performance" + 0.008*"particle" + 0.007*"node" + 0.007*"mesh" + 0.007*

In [35]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [134]:
textiwant.replace('\n', ' ')

NameError: name 'textiwant' is not defined